# 工具和路由 Tools and Routing

 - [一、设置OpenAI API Key](#一、设置OpenAI-API-Key)
 - [二、通过Langchain定义工具](#二、通过Langchain定义工具)
     - [2.1 通过装饰器之间定义Tool](#2.1-通过装饰器之间定义Tool)
     - [2.2 通过pydantic类定义Tool](#2.2-通过pydantic类定义Tool)
     - [2.3 天气查询应用案例](#2.3-天气查询应用案例)
     - [2.4 通过tool定义function](#2.4-通过tool定义function)
     - [2.5 通过API定义function案例](#2.5-通过API定义function案例)
 - [三、 路由](#三、路由)
     - [3.1 Tool转换为Function](#3.1-Tool转换为Function)
     - [3.2 通过route进行tools的选择](#3.2-通过route进行tools的选择)
     - [3.3 输出解析器](#3.3-输出解析器)
 - [四、总结](#四、总结)


# 一、设置OpenAI-API-Key

详细内容见`设置OpenAI_API_KEY.ipynb`文件

In [1]:
import os

from dotenv import load_dotenv, find_dotenv

loaded = load_dotenv(find_dotenv(), override=True)
# 从环境变量中获取 OpenAI API Key 或者直接赋值
API_KEY = os.getenv("API_KEY")

# 如果您使用的是官方 API，就直接用 https://api.siliconflow.cn/v1 就行。
BASE_URL = "https://api.siliconflow.cn/v1"

# 二、通过Langchain定义工具

## 2.1-通过装饰器之间定义Tool

In [2]:
from langchain.agents import tool

# tool装饰器包装了search函数
@tool
def search(query: str) -> str:
    """在网络上查询天气"""
    return "42度"

In [3]:
# 搜索工具的函数名
print(search.name)
#搜索工具的功能描述（即函数注释）
print(search.description)
# 搜索工具需要传递的参数
print(search.args)

search
在网络上查询天气
{'query': {'title': 'Query', 'type': 'string'}}


## 2.2-通过pydantic类定义Tool

In [4]:
# 导入 Pydantic 库中的 BaseModel 类和 Field 函数，它们用于定义数据模型和字段。
from pydantic import BaseModel, Field

class SearchInput(BaseModel):
    """
    定义了 SearchInput 类中的一个属性 query，它是一个字符串类型。通过 Field 函数，你为这个字段提供了一些配置
    其中 description 参数用于描述这个字段的用途，即 "Thing to search for"（要搜索的内容）。
    """
    query: str = Field(description="你需要搜索的东西")

In [5]:
# 搜索工具类需要传递的参数
search.args

{'query': {'title': 'Query', 'type': 'string'}}

In [6]:
# args_schema参数传递SearchInput工具类
@tool(args_schema=SearchInput)
def search_zh(query: str) -> str:
    """在网上查找温度"""
    return "42度"

In [7]:
search.args

{'query': {'title': 'Query', 'type': 'string'}}

In [8]:
search.run("圣弗朗西斯科")

'42度'

## 2.3-天气查询应用案例
整体代码逻辑：
1. 使用 Pydantic 定义了输入类OpenMeteoInput，以及输入的两个参数（经度和纬度）的输入格式
2. 定义了一个函数 get_current_temperature，该函数使用 OpenMeteo API 获取给定坐标位置的当前温度。
3. get_current_temperature函数通过发送 HTTP 请求获取 API 响应，然后从响应中提取并计算出当前时间对应的温度。
4. get_current_temperature函数返回一个字符串，其中包含了当前温度的信息。

In [9]:
# 导入所需的库
import requests
from pydantic import BaseModel, Field
import datetime

# 定义输入类（input schema）
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="要获取天气数据的位置的纬度") 
    longitude: float = Field(..., description="要获取天气数据的位置的经度") 

# 使用 @tool 装饰器并指定输入模型
@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """"获取给定坐标的温度"""
    
    # Open Meteo API 的URL
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    
    # 请求参数
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    # 发送 API 请求
    response = requests.get(BASE_URL, params=params)
    
    # 检查响应状态码
    if response.status_code == 200:
        # 解析 JSON 响应
        results = response.json()
    else:
        # 处理请求失败的情况
        raise Exception(f"API Request failed with status code: {response.status_code}")

    # 获取当前 UTC 时间
    current_utc_time = datetime.datetime.now(datetime.UTC)
    
    # 将时间字符串转换为 datetime 对象
    time_list = [datetime.datetime.fromisoformat(time_str).replace(tzinfo=datetime.timezone.utc) for time_str in results['hourly']['time']]
    
    # 获取温度列表
    temperature_list = results['hourly']['temperature_2m']
    
    # 找到最接近当前时间的索引
    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    
    # 获取当前温度
    current_temperature = temperature_list[closest_time_index]
    
    # 返回当前温度的字符串形式
    return f'当前的温度是 {current_temperature}°C'


In [10]:
# 工具的名字
print(get_current_temperature.name)
# 工具的功能描述
print(get_current_temperature.description)
# 工具的输入参数
print(get_current_temperature.args)

get_current_temperature
"获取给定坐标的温度
{'latitude': {'description': '要获取天气数据的位置的纬度', 'title': 'Latitude', 'type': 'number'}, 'longitude': {'description': '要获取天气数据的位置的经度', 'title': 'Longitude', 'type': 'number'}}


## 2.4-通过tool定义function

In [11]:
# 导入openai的模板
from langchain_core.utils.function_calling import convert_to_openai_function

In [12]:
# 将定义好的工具直接传入模板，打印tool的名字、描述和输入参数格式
convert_to_openai_function(get_current_temperature)

{'name': 'get_current_temperature',
 'description': '"获取给定坐标的温度',
 'parameters': {'properties': {'latitude': {'description': '要获取天气数据的位置的纬度',
    'type': 'number'},
   'longitude': {'description': '要获取天气数据的位置的经度', 'type': 'number'}},
  'required': ['latitude', 'longitude'],
  'type': 'object'}}

In [13]:
# 调用工具
get_current_temperature.invoke({"latitude": 13, "longitude": 14})

'当前的温度是 39.7°C'

In [14]:
import wikipedia

# 定义维基百科搜索的tool
@tool
def search_wikipedia(query: str) -> str:
    """打开维基百科搜索并获得页面的摘要"""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]: #取前三个页面标题
        try:
            #使用 wikipedia 模块的 page 函数，获取指定标题的维基百科页面对象。
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False) 
            # 获取页面摘要
            summaries.append(f"页面: {page_title}\n摘要: {wiki_page.summary}")
        except (
            wikipedia.exceptions.PageError,
            wikipedia.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "维基百科没有搜索到合适的结果"
    return "\n\n".join(summaries)

In [15]:
# 工具的名字
search_wikipedia.name

'search_wikipedia'

In [16]:
# 工具的描述
search_wikipedia.description

'打开维基百科搜索并获得页面的摘要'

In [17]:
# 将工具格式化为 OpenAI 函数
convert_to_openai_function(search_wikipedia)

{'name': 'search_wikipedia',
 'description': '打开维基百科搜索并获得页面的摘要',
 'parameters': {'properties': {'query': {'type': 'string'}},
  'required': ['query'],
  'type': 'object'}}

In [19]:
# 调用
search_wikipedia.invoke({"query": "langchain"})

'页面: LangChain\n摘要: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain\'s use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.\n\n页面: Retrieval-augmented generation\n摘要: Retrieval-augmented generation (RAG) is a technique that enables large language models (LLMs) to retrieve and incorporate new information. With RAG, LLMs do not respond to user queries until they refer to a specified set of documents. These documents supplement information from the LLM\'s pre-existing training data. This allows LLMs to use domain-specific and/or updated information that is not available in the training data. For example, this helps LLM-based chatbots access internal company data or generate responses based on authoritative sources.\nRAG improves large language models (LLMs) by incorporating

## 2.5-通过API定义function案例

In [20]:
# openapi_spec_to_openai_fn可以把json格式的API定义转换成openai的function call格式
from langchain.chains.openai_functions.openapi import openapi_spec_to_openai_fn
from langchain.chains.openai_tools.openapi import openapi_spec_to_openai_fn

# OpenAPISpec是标准化的API格式定义
from langchain.utilities.openapi import OpenAPISpec

In [21]:
# json格式的API定义
text = """
{
  "openapi": "3.1.0",
  "info": {
    "version": "1.0.0",
    "title": "Swagger Petstore",
    "license": {
      "name": "MIT"
    }
  },
  "servers": [
    {
      "url": "http://petstore.swagger.io/v1"
    }
  ],
  "paths": {
    "/pets": {
      "get": {
        "summary": "List all pets",
        "operationId": "listPets",
        "tags": [
          "pets"
        ],
        "parameters": [
          {
            "name": "limit",
            "in": "query",
            "description": "How many items to return at one time (max 100)",
            "required": false,
            "schema": {
              "type": "integer",
              "maximum": 100,
              "format": "int32"
            }
          }
        ],
        "responses": {
          "200": {
            "description": "A paged array of pets",
            "headers": {
              "x-next": {
                "description": "A link to the next page of responses",
                "schema": {
                  "type": "string"
                }
              }
            },
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Pets"
                }
              }
            }
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      },
      "post": {
        "summary": "Create a pet",
        "operationId": "createPets",
        "tags": [
          "pets"
        ],
        "responses": {
          "201": {
            "description": "Null response"
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      }
    },
    "/pets/{petId}": {
      "get": {
        "summary": "Info for a specific pet",
        "operationId": "showPetById",
        "tags": [
          "pets"
        ],
        "parameters": [
          {
            "name": "petId",
            "in": "path",
            "required": true,
            "description": "The id of the pet to retrieve",
            "schema": {
              "type": "string"
            }
          }
        ],
        "responses": {
          "200": {
            "description": "Expected response to a valid request",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Pet"
                }
              }
            }
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      }
    }
  },
  "components": {
    "schemas": {
      "Pet": {
        "type": "object",
        "required": [
          "id",
          "name"
        ],
        "properties": {
          "id": {
            "type": "integer",
            "format": "int64"
          },
          "name": {
            "type": "string"
          },
          "tag": {
            "type": "string"
          }
        }
      },
      "Pets": {
        "type": "array",
        "maxItems": 100,
        "items": {
          "$ref": "#/components/schemas/Pet"
        }
      },
      "Error": {
        "type": "object",
        "required": [
          "code",
          "message"
        ],
        "properties": {
          "code": {
            "type": "integer",
            "format": "int32"
          },
          "message": {
            "type": "string"
          }
        }
      }
    }
  }
}
"""

In [22]:
# 从text中导入API的详细定义
spec = OpenAPISpec.from_text(text)

In [23]:
# 转换成openai的fuction call格式
pet_openai_functions, pet_callables = openapi_spec_to_openai_fn(spec)

In [24]:
# 查看fuction的定义
pet_openai_functions

[{'name': 'listPets',
  'description': 'List all pets',
  'parameters': {'type': 'object',
   'properties': {'params': {'type': 'object',
     'properties': {'limit': {'type': 'integer',
       'maximum': 100.0,
       'schema_format': 'int32',
       'description': 'How many items to return at one time (max 100)'}},
     'required': []}}}},
 {'name': 'createPets',
  'description': 'Create a pet',
  'parameters': {'type': 'object', 'properties': {}}},
 {'name': 'showPetById',
  'description': 'Info for a specific pet',
  'parameters': {'type': 'object',
   'properties': {'path_params': {'type': 'object',
     'properties': {'petId': {'type': 'string',
       'description': 'The id of the pet to retrieve'}},
     'required': ['petId']}}}}]

In [25]:
# 导入模型
from langchain_openai.chat_models import ChatOpenAI

In [26]:
# 设置模型温度系数并传入function
model = ChatOpenAI(temperature=0, model_name="deepseek-ai/DeepSeek-V3", max_tokens=4096,
                        openai_api_key=API_KEY, openai_api_base=BASE_URL, max_retries=3,
                        seed=42, presence_penalty=0.1, frequency_penalty=0.1,
                        extra_body={
                            "enable_thinking": False
                        }
                        )
pet_openai_tools = [{"type": "function", "function": x} for x in pet_openai_functions]
model = model.bind(tools=pet_openai_tools)

In [27]:
# 输入query，查看模型调用的function以及返回信息
model.invoke("这三只宠物的名字叫什么？")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '019763bd2aa9a9e26cadc34c598bd1ea', 'function': {'arguments': '{"params":{"limit":3}}', 'name': 'listPets'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 269, 'total_tokens': 277, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek-ai/DeepSeek-V3', 'system_fingerprint': '', 'id': '019763bd23aec9eca496ca05864c7441', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--1efbacc2-2874-42a7-ad18-99abb03ff774-0', tool_calls=[{'name': 'listPets', 'args': {'params': {'limit': 3}}, 'id': '019763bd2aa9a9e26cadc34c598bd1ea', 'type': 'tool_call'}], usage_metadata={'input_tokens': 269, 'output_tokens': 8, 'total_tokens': 277, 'input_token_details': {}, 'output_token_details': {}})

In [28]:
model.invoke("告诉我id为42的宠物的消息")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '019763bde509912c0143f3040232d1cc', 'function': {'arguments': '{"path_params":{"petId":"42"}}', 'name': 'showPetById'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 270, 'total_tokens': 281, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek-ai/DeepSeek-V3', 'system_fingerprint': '', 'id': '019763bdd8af21210153ac67e1323bf6', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--1599c3ff-eb5f-4baa-b2de-0ba9fd437de1-0', tool_calls=[{'name': 'showPetById', 'args': {'path_params': {'petId': '42'}}, 'id': '019763bde509912c0143f3040232d1cc', 'type': 'tool_call'}], usage_metadata={'input_tokens': 270, 'output_tokens': 11, 'total_tokens': 281, 'input_token_details': {}, 'output_token_details': {}})

# 3、 路由

展示一个函数调用的例子，用于在两个候选函数之间做出决策。



## 3.1-Tool转换为Function

鉴于我们上面提到的工具，让我们将它们格式化为 OpenAI 函数，并展示相同的行为。

In [33]:
# 将工具格式化为 OpenAI 函数
functions = [
    convert_to_openai_function(f) for f in [
        search_wikipedia, get_current_temperature
    ]
]

tools = [{"type": "function", "function": x} for x in functions]

model = model.bind(tools=tools)

In [34]:
# 模型调用
model.invoke("圣佛朗西斯科现在的温度是多少？")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '019763c07bc9b62881b184fd27e348a8', 'function': {'arguments': '{"latitude":37.7749,"longitude":-122.4194}', 'name': 'get_current_temperature'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 202, 'total_tokens': 220, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek-ai/DeepSeek-V3', 'system_fingerprint': '', 'id': '019763c073b16a942f51e4528fc8b891', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--66280092-fb55-4dea-bb53-e9c70ff8f138-0', tool_calls=[{'name': 'get_current_temperature', 'args': {'latitude': 37.7749, 'longitude': -122.4194}, 'id': '019763c07bc9b62881b184fd27e348a8', 'type': 'tool_call'}], usage_metadata={'input_tokens': 202, 'output_tokens': 18, 'total_tokens': 220, 'input_token_details': {}, 'output_token_details': {}})

In [35]:
# 模型调用
model.invoke("什么是langchain？")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '019763c09f9afffa19c833f910b25c30', 'function': {'arguments': '{"query":"langchain"}', 'name': 'search_wikipedia'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 197, 'total_tokens': 204, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek-ai/DeepSeek-V3', 'system_fingerprint': '', 'id': '019763c096e763b61be53a675be45171', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--7e8453dd-2ab0-49a1-8305-dd53964251cf-0', tool_calls=[{'name': 'search_wikipedia', 'args': {'query': 'langchain'}, 'id': '019763c09f9afffa19c833f910b25c30', 'type': 'tool_call'}], usage_metadata={'input_tokens': 197, 'output_tokens': 7, 'total_tokens': 204, 'input_token_details': {}, 'output_token_details': {}})

In [36]:
# 使用template构造prompt
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "你是个乐于助人的助理"),
    ("user", "{input}"),
])

# 创建处理链，将 prompt和model连接起来
chain = prompt | model

In [37]:
# 输入query进行调用
chain.invoke({"input": "圣佛朗西斯科现在的温度是多少？"})

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '019763c11430594f4232189db22ff678', 'function': {'arguments': '{"latitude":37.7749,"longitude":-122.4194}', 'name': 'get_current_temperature'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 209, 'total_tokens': 227, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek-ai/DeepSeek-V3', 'system_fingerprint': '', 'id': '019763c10b7ca3ad862c175b78ebccdf', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--3f6bcb5b-a363-4c6e-9c7c-8e95fe6aa956-0', tool_calls=[{'name': 'get_current_temperature', 'args': {'latitude': 37.7749, 'longitude': -122.4194}, 'id': '019763c11430594f4232189db22ff678', 'type': 'tool_call'}], usage_metadata={'input_tokens': 209, 'output_tokens': 18, 'total_tokens': 227, 'input_token_details': {}, 'output_token_details': {}})

In [38]:
# 导入输出解析的包
from langchain.agents.output_parsers import ToolsAgentOutputParser

In [39]:
# 创建处理链，将 prompt、model 和 ToolsAgentOutputParser 连接起来
chain = prompt | model | ToolsAgentOutputParser()

In [40]:
# 调用
result = chain.invoke({"input": "圣佛朗西斯科现在的温度是多少？"})

In [41]:
# 打印返回的类型，可以判断是否产生function的调用
type(result)

list

In [44]:
# 查看调用的tool
result[0].tool

'get_current_temperature'

In [45]:
# 查看tool的输入，result.message_log可以查看调用结果
result[0].tool_input

{'latitude': 37.7749, 'longitude': -122.4194}

In [46]:
# 调用的获取温度的工具
get_current_temperature(result[0].tool_input)

'当前的温度是 11.6°C'

In [47]:
# 继续调用
result = chain.invoke({"input": "你好!"})

In [49]:
# 打印返回的类型，可以判断是否产生function的调用
type(result)

langchain_core.agents.AgentFinish

In [50]:
# 查看返回值
result.return_values

{'output': '你好！有什么可以帮您的吗？'}

## 3.2-通过route进行tools的选择

In [62]:
"""
route会根据result进行tools的选择：
AgentFinish：表示已经完成，可以输出
AgentActionMessageLog：表示未完成，需要继续进行route调用tools
"""
from langchain.schema.agent import AgentFinish

def route(result):
    if isinstance(result, AgentFinish):
        return result.return_values['output']
    else:
        tools = {
            "search_wikipedia": search_wikipedia, 
            "get_current_temperature": get_current_temperature,
        }
        return tools[result[0].tool].run(result[0].tool_input)

In [63]:
chain = prompt | model | ToolsAgentOutputParser() | route

In [57]:
chain.invoke({"input": "你好！"})

'你好！有什么可以帮您的吗？'

In [58]:
chain.invoke({"input": "圣弗朗西斯科的天气现在怎么样？"})

'圣弗朗西斯科（旧金山）的天气可能会因季节和时间而变化。为了获取准确的当前天气信息，我需要知道具体的经纬度坐标。如果你能提供这些信息，我可以帮你查询；或者，你也可以告诉我一个具体的地标或区域，我可以尝试找到相关的天气数据。'

In [64]:
chain.invoke({"input": "什么是langchain?"})

'页面: LangChain\n摘要: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain\'s use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.\n\n页面: Retrieval-augmented generation\n摘要: Retrieval-augmented generation (RAG) is a technique that enables large language models (LLMs) to retrieve and incorporate new information. With RAG, LLMs do not respond to user queries until they refer to a specified set of documents. These documents supplement information from the LLM\'s pre-existing training data. This allows LLMs to use domain-specific and/or updated information that is not available in the training data. For example, this helps LLM-based chatbots access internal company data or generate responses based on authoritative sources.\nRAG improves large language models (LLMs) by incorporating

# 四、英文版提示

我们总结一下完整的调用流程：

构造Prompt --> 调用模型 --> 解析模型返回的结果 --> 进行路由选择对应的tool


In [65]:
@tool
def search(query: str) -> str:
    """"Search for weather online"""
    return "42f"

In [66]:
# 搜索工具的函数名
print(search.name)
#搜索工具的功能描述（即函数注释）
print(search.description)
# 搜索工具需要传递的参数
print(search.args)

search
"Search for weather online
{'query': {'title': 'Query', 'type': 'string'}}


In [67]:
from pydantic import BaseModel, Field

class SearchInput(BaseModel):
    query: str = Field(description="Thing to search for")

In [68]:
search.args

{'query': {'title': 'Query', 'type': 'string'}}

In [69]:
search.run("sf")

'42f'

In [70]:
# 导入所需的库
import requests
from pydantic import BaseModel, Field
import datetime

# 定义输入类（input schema）
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for") #要获取天气数据的位置的纬度
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for") #要获取天气数据的位置的经度

# 使用 @tool 装饰器并指定输入模型
@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """"Fetch current temperature for given coordinates."""
    
    # Open Meteo API 的URL
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    
    # 请求参数
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    # 发送 API 请求
    response = requests.get(BASE_URL, params=params)
    
    # 检查响应状态码
    if response.status_code == 200:
        # 解析 JSON 响应
        results = response.json()
    else:
        # 处理请求失败的情况
        raise Exception(f"API Request failed with status code: {response.status_code}")

    # 获取当前 UTC 时间
    current_utc_time = datetime.datetime.now(datetime.UTC)
    
    # 将时间字符串转换为 datetime 对象
    time_list = [datetime.datetime.fromisoformat(time_str).replace(tzinfo=datetime.timezone.utc) for time_str in results['hourly']['time']]
    # 获取温度列表
    temperature_list = results['hourly']['temperature_2m']
    
    # 找到最接近当前时间的索引
    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    
    # 获取当前温度
    current_temperature = temperature_list[closest_time_index]
    
    # 返回当前温度的字符串形式
    return f'The current temperature is {current_temperature}°C'


In [71]:
# 工具的名字
print(get_current_temperature.name)
# 工具的功能描述
print(get_current_temperature.description)
# 工具的输入参数
print(get_current_temperature.args)

get_current_temperature
"Fetch current temperature for given coordinates.
{'latitude': {'description': 'Latitude of the location to fetch weather data for', 'title': 'Latitude', 'type': 'number'}, 'longitude': {'description': 'Longitude of the location to fetch weather data for', 'title': 'Longitude', 'type': 'number'}}


In [72]:
# 导入openai的模板
from langchain_core.utils.function_calling import convert_to_openai_function

convert_to_openai_function(get_current_temperature)

{'name': 'get_current_temperature',
 'description': '"Fetch current temperature for given coordinates.',
 'parameters': {'properties': {'latitude': {'description': 'Latitude of the location to fetch weather data for',
    'type': 'number'},
   'longitude': {'description': 'Longitude of the location to fetch weather data for',
    'type': 'number'}},
  'required': ['latitude', 'longitude'],
  'type': 'object'}}

In [73]:
get_current_temperature({"latitude": 13, "longitude": 14})

'The current temperature is 39.7°C'

In [74]:
import wikipedia

# 定义维基百科搜索的tool
@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]: #取前三个页面标题
        try:
            #使用 wikipedia 模块的 page 函数，获取指定标题的维基百科页面对象。
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False) 
            # 获取页面摘要
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            wikipedia.exceptions.PageError,
            wikipedia.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)

In [75]:
# 将工具格式化为 OpenAI 函数
convert_to_openai_function(search_wikipedia)

{'name': 'search_wikipedia',
 'description': 'Run Wikipedia search and get page summaries.',
 'parameters': {'properties': {'query': {'type': 'string'}},
  'required': ['query'],
  'type': 'object'}}

In [77]:
from langchain_openai.chat_models import ChatOpenAI
# 将工具格式化为 OpenAI 函数

functions = [
    convert_to_openai_function(f) for f in [
        search_wikipedia, get_current_temperature
    ]
]

model = ChatOpenAI(temperature=0, model_name="deepseek-ai/DeepSeek-V3", max_tokens=4096,
                        openai_api_key=API_KEY, openai_api_base=BASE_URL, max_retries=3,
                        seed=42, presence_penalty=0.1, frequency_penalty=0.1,
                        extra_body={
                            "enable_thinking": False
                        }
                        )

pet_openai_tools = [{"type": "function", "function": x} for x in functions]

model = model.bind(tools=pet_openai_tools)

In [78]:
# 使用template构造prompt
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
])

# 创建处理链，将 prompt和model连接起来
chain = prompt | model

In [84]:
# 导入输出解析的包
from langchain.agents.output_parsers import ToolsAgentOutputParser

chain = prompt | model | ToolsAgentOutputParser()

In [85]:
# 调用
chain.invoke({"input": "what is the weather in sf right now"})

[ToolAgentAction(tool='get_current_temperature', tool_input={'latitude': 37.7749, 'longitude': -122.4194}, log="\nInvoking: `get_current_temperature` with `{'latitude': 37.7749, 'longitude': -122.4194}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '019763cd4af18352cf313474a6fabd16', 'function': {'arguments': '{"latitude":37.7749,"longitude":-122.4194}', 'name': 'get_current_temperature'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 214, 'total_tokens': 232, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek-ai/DeepSeek-V3', 'system_fingerprint': '', 'id': '019763cd43d1008dc26c808016db4128', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--a6208f75-6fd3-4bf2-8e8d-31879216d16b-0', tool_calls=[{'name': 'get_current_temperature', 'args': {'latitude': 37.7749, 'longitude': -122.4194}, 'id': '019763

In [86]:
# 调用并查看结果
chain.invoke({"input": "hi!"})

AgentFinish(return_values={'output': 'Well, hello there! What can I do for you today? Or are we just exchanging pleasantries? 😏'}, log='Well, hello there! What can I do for you today? Or are we just exchanging pleasantries? 😏')

In [87]:
# 调用
result = chain.invoke({"input": "langchain?"})
result

AgentFinish(return_values={'output': "Ah, LangChain! The sassy assistant is here to enlighten you. \n\n**LangChain** is a framework designed to simplify the development of applications that leverage large language models (LLMs). It provides tools and abstractions to make it easier to build, customize, and deploy LLM-powered applications. Think of it as a Swiss Army knife for working with AI models like GPT-3, GPT-4, and others.\n\n### Key Features of LangChain:\n1. **Chains**: Combine multiple steps or calls to LLMs into a single workflow (e.g., question-answering, summarization).\n2. **Agents**: Let the LLM decide what actions to take (like using tools or APIs) to solve a task.\n3. **Memory**: Add statefulness to your applications (e.g., chatbots that remember past interactions).\n4. **Document Loaders & Indexes**: Easily work with external data sources (PDFs, websites, etc.).\n5. **Customizability**: Fine-tune and extend the framework to fit your needs.\n\n### Why Use LangChain?\n- *

In [88]:
# 调用
result = chain.invoke({"input": "What is the weather in san francisco right now?"})
result

[ToolAgentAction(tool='search_wikipedia', tool_input={'query': 'San Francisco coordinates'}, log="\nInvoking: `search_wikipedia` with `{'query': 'San Francisco coordinates'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '019763cdeea66df09dc6ae375c7e42a6', 'function': {'arguments': '{"query":"San Francisco coordinates"}', 'name': 'search_wikipedia'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 218, 'total_tokens': 226, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek-ai/DeepSeek-V3', 'system_fingerprint': '', 'id': '019763cde6fea46a566f99ff1fb9f304', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--7ac52ea5-fd7d-4e52-a0a4-0fdc4baa0528-0', tool_calls=[{'name': 'search_wikipedia', 'args': {'query': 'San Francisco coordinates'}, 'id': '019763cdeea66df09dc6ae375c7e42a6', 'type': 'tool_call'}], usa

In [89]:
# 导入tool包
from langchain.agents import tool

In [90]:
from langchain.schema.agent import AgentFinish
def route(result):
    if isinstance(result, AgentFinish):
        return result.return_values['output']
    else:
        tools = {
            "search_wikipedia": search_wikipedia, 
            "get_current_temperature": get_current_temperature,
        }
        return tools[result[0].tool].run(result[0].tool_input)

In [91]:
chain = prompt | model | ToolsAgentOutputParser() | route

In [92]:
# 调用并查看结果
chain.invoke({"input": "hi!"})

'Well, hello there! What can I do for you today? Or are you just here to bask in my sassy brilliance? 😏'

In [93]:
# 调用
result = chain.invoke({"input": "What is langchain?"})
result

"LangChain is a framework designed to simplify the development of applications that leverage large language models (LLMs) like OpenAI's GPT. It provides tools and abstractions to help developers build, chain, and manage interactions with LLMs, making it easier to create complex workflows, integrate external data sources, and maintain stateful conversations.\n\n### Key Features of LangChain:\n1. **Chaining**: Allows you to chain multiple LLM calls or other operations together to create more sophisticated applications.\n2. **Memory**: Supports maintaining context or memory across interactions, which is useful for chatbots or multi-step workflows.\n3. **Data Augmentation**: Enables integration with external data sources (like databases or APIs) to provide richer responses.\n4. **Agents**: Provides a way to build agents that can dynamically decide which actions to take based on user input.\n5. **Customization**: Offers flexibility to customize prompts, models, and workflows to suit specifi

In [94]:
# 调用
result = chain.invoke({"input": "What is the weather in san francisco right now?"})
result

"Page: Mission District, San Francisco\nSummary: The Mission District (Spanish: Distrito de la Misión), commonly known as the Mission (Spanish: La Misión), is a neighborhood in San Francisco, California. One of the oldest neighborhoods in San Francisco, the Mission District's name is derived from Mission San Francisco de Asís, built in 1776 by the Spanish. The Mission is historically one of the most notable centers of the city's Hispanic community.\n\nPage: Presidio of San Francisco\nSummary: The Presidio of San Francisco (originally, El Presidio Real de San Francisco or The Royal Fortress of Saint Francis) is a park and former U.S. Army post on the northern tip of the San Francisco Peninsula in San Francisco, California, and is part of the Golden Gate National Recreation Area.\nIt had been a fortified location since September 17, 1776, when New Spain established the presidio to gain a foothold in Alta California and the San Francisco Bay. It passed to Mexico in 1820, which in turn pas